In [14]:
import os
import math

import time
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm as cm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.covariance import EllipticEnvelope
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

In [15]:
_train_=pd.read_csv('../input/train.csv')
places=_train_['place_id'].unique()

_train_.sort(['time'], inplace=True)
trainRatio=0.7
train=_train_[:int(len(_train_)*trainRatio)]
train=train[train['accuracy']>2]

placeCounts=train['place_id'].value_counts()



valid=_train_[int(len(_train_)*trainRatio):]

/home/zhenjie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [16]:
train=train[(placeCounts[train['place_id']]>10).values]

In [17]:
def featureFactory(dataFrame):
    dataFrame.loc[:,'day']=dataFrame.loc[:, 'time'].div(1440).map(int)
    dataFrame.loc[:,'weekday']=dataFrame.loc[:,'day']%7
    dataFrame.loc[:,'year']=dataFrame.loc[:,'day'].div(365).map(int)
    dataFrame.loc[:,'month']=(dataFrame.loc[:,'day']%365).div(30).map(int)
    dataFrame.loc[:,'hour']=dataFrame.loc[:,'time'].div(60).map(int)
    dataFrame.loc[:, 'hourInDay']=dataFrame.loc[:,'hour']%24
    
    dataFrame.loc[:, 'originalIndex']=xrange(len(dataFrame))

    
featureFactory(train)
featureFactory(valid)


In [18]:
test=pd.read_csv('../input/test.csv')
featureFactory(test)

In [19]:
featureFactory(test)

In [20]:
g=train.groupby('place_id')
places=pd.DataFrame()
places['xMean']=g['x'].mean()
places['xStd']=g['x'].std()

places['yMean']=g['y'].mean()
places['yStd']=g['y'].std()

places['hourInDayMean']=g['hourInDay'].mean()
places['hourInDayStd']=g['hourInDay'].std()

places['monthMean']=g['month'].mean()
places['monthStd']=g['month'].std()

places['dayMean']=g['day'].mean()
places['dayStd']=g['day'].std()

places.head()

sTime=time.time()

train=train[(abs(train['y']-places.loc[train['place_id'], 'yMean'].values)<5*places.loc[train['place_id'], 'yStd'].values)\
         & (abs(train['x']-places.loc[train['place_id'], 'xMean'].values)<5*places.loc[train['place_id'], 'xStd'].values)\
           
           ]

print time.time()-sTime

,xMean,xStd,yMean,yStd,hourInDayMean,hourInDayStd,monthMean,monthStd,dayMean,dayStd
place_id,,,,,,,,,,
1000015801,2.689782,0.168262,5.550973,0.014583,16.183099,5.524775,3.323944,3.003687,129.704225,103.774948
1000017288,7.327345,0.338355,4.346280,0.010608,12.743902,4.147916,4.841463,3.342467,185.317073,110.229713
1000025138,0.987664,0.043769,5.570396,0.010413,12.619952,3.580766,5.674584,3.766525,233.244656,110.803449
1000052096,2.846386,0.443136,5.833500,0.011343,9.780029,3.357571,4.534009,3.555349,184.413893,118.278274
1000063498,3.867737,1.084438,7.547868,0.022191,16.684211,2.688507,7.052632,3.487656,266.210526,93.450866


116.685582161


In [31]:
nXRegions=50
nYRegions=50
xMin=0
xMax=10.1
yMin=0
yMax=10.1
nXRegions=nXRegions*int(xMax-xMin)/10
nYRegions=nYRegions*int(yMax-yMin)/10
train=train[(train['x']<xMax) & (train['x']>=xMin) & (train['y']<yMax) & (train['y']>=yMin)]
valid=valid[(valid['x']<xMax) & (valid['x']>=xMin) & (valid['y']<yMax) & (valid['y']>=yMin)]


f=['x','y','accuracy', 'time', 'weekday', 'hourInDay', 'month', 'year']
print len(valid)

8735407


In [22]:
(valid['day'].div(30)+4).describe()

count    8.735407e+06
mean     1.990981e+01
std      1.341564e+00
min      1.756667e+01
25%      1.873333e+01
50%      1.993333e+01
75%      2.110000e+01
max      2.216667e+01
Name: day, dtype: float64

In [23]:
# def regionalForestPred(pred, classes, regionalPoints, data):

    
#     prob=[sorted(zip(p, classes))[-3:][::-1] for p in pred]
    
    
#     regionalPred=[]
#     for i in xrange(len(data)):
#         sampleY=data['y'].iloc[i]
#         sampleX=data['x'].iloc[i]
#         sampleDay=data['day'].iloc[i]

#         filteredPlaces=filter(lambda x: \
#                               sampleY<=yMaxMap[x[1]]+yAllowance and sampleY>=yMinMap[x[1]]-yAllowance and \
#                               sampleX<=xMaxMap[x[1]] and sampleX>=xMinMap[x[1]] and \
#                               sampleDay<=dayMaxMap[x[1]] and sampleDay>=dayMinMap[x[1]], prob[i])
#         if filteredPlaces:
#             filteredPlaces=map(lambda x: [x[0]*regionalPoints, x[1]], filteredPlaces)
#         regionalPred.append(filteredPlaces)
#     return regionalPred

In [52]:
def train_test(modelSerialization=False, resultSerialization=False, model='rf', predictSet=valid):
    
    

    if modelSerialization or resultSerialization:
        base='./'
        folderName=model+'-'+time.strftime('%c')
        folderPath=base+folderName+'/'
        if not os.path.exists(folderPath):
            os.makedirs(folderPath)
            
        
    count=0
    accuracies=[]
    FBscores=[]
    
    startTime=time.time()
    trainingTimes=[]
    predictionTimes=[]
    
    testNum=0

    roundNum=5
    for xNum, yNum in ((xn, yn) for xn in xrange(nXRegions) for yn in xrange(nYRegions)):

        xStep=round(1.0*(xMax-xMin)/nXRegions,roundNum)
        xStart=round(xNum*xStep,roundNum)
        xEnd=round(xStart+xStep,roundNum)

        yStep=round(1.0*(yMax-yMin)/nYRegions,roundNum)
        yStart=round(yNum*yStep,roundNum)
        yEnd=round(yStart+yStep,roundNum)

        regionalTrain=train[(train['x']<xEnd) & (train['x']>=xStart) & (train['y']<yEnd) & (train['y']>=yStart)]
#         regionalTrain['sampleWeight']=

        if len(regionalTrain):
            regionalPredictSet=predictSet[(predictSet['x']<xEnd) & (predictSet['x']>=xStart) & (predictSet['y']<yEnd) & (predictSet['y']>=yStart)]
            testNum+=len(regionalPredictSet)
            
            s_w_rf=1.2**(15-regionalTrain['day'].values/30.0)
#             s_w_rf=62-regionalTrain['day'].values/7.0

#             s_w=1**(15-regionalTrain['day'].values/30.0)
            
            trainingStartTime=time.time()
#             clf=RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features=None).fit(regionalTrain[f], regionalTrain['place_id'])   
#             clf=RandomForestClassifier(n_jobs=-1, n_estimators=1000, random_state=0).fit(regionalTrain[f], regionalTrain['place_id'], sample_weight=s_w_rf)  
            clf=RandomForestClassifier(n_jobs=-1, n_estimators=300, random_state=0).fit(regionalTrain[f], regionalTrain['place_id'], sample_weight=s_w_rf)  

#             clf=XGBClassifier(learning_rate=0.02, n_estimators=250, objective='multi:softprob', max_depth=3, seed=0).fit(regionalTrain[f], regionalTrain['place_id'], sample_weight=s_w)

            trainingTimes.append(time.time()-trainingStartTime)
            
            
            if modelSerialization:
                modelFileName='{:04d}-{}-{}-{}-{}.clf'.format(count, xStart, xEnd, yStart, yEnd)
                modelFilePath=folderPath+modelFileName
                with open(modelFilePath, 'ab+') as fo:
                    pickle.dump(clf, fo, pickle.HIGHEST_PROTOCOL)
            
            
            predictionStartTime=time.time()
            prob=clf.predict_proba(regionalPredictSet[f])
            predictionTimes.append(time.time()-predictionStartTime)
            
            pred=[sorted(zip(p, clf.classes_))[-3:][::-1] for p in prob]
            prediction=[zip(*p)[1] for p in pred]
            confidence=[zip(*p)[0] for p in pred]
            
            
                
            fbscoreForSerialization=-1
            if 'place_id' in predictSet.columns:
                regionalAccuracy=1.0*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                regionalConfidence=[sum(z)/len(regionalPredictSet) for z in zip(*confidence)]
                regionalConfidence3=sum(regionalConfidence)

                fbAccuracy=0
                fbAccuracy+=1.0*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][:1] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                fbAccuracy+=1.0/2*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][1:2] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                fbAccuracy+=1.0/3*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][2:3] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                FBscores.append(fbAccuracy)
                print 'region {}: {},{} accuracy: {},  fbAccu: {}, confidence: {}:'.format(count, xNum, yNum, regionalAccuracy, fbAccuracy, regionalConfidence3)
                accuracies.append(regionalAccuracy)
                if resultSerialization:
                    fbscoreForSerialization=fbAccuracy
                
            if resultSerialization:
                resultFileName='{:04d}-{}-{}-{}-{}.rst'.format(count, xStart, xEnd, yStart, yEnd)
                resultFilePath=folderPath+resultFileName
                predColumns=zip(*prediction)
                confColumns=zip(*confidence)

                
                if len(predColumns[2])!=len(prediction):
                    print 'missing values',count, len(predColumns[0])
                
                results=pd.DataFrame({'originalIndex': regionalPredictSet['originalIndex'].tolist(),\
                                      'x':regionalPredictSet['x'].tolist(), 'y':regionalPredictSet['y'].tolist(), \
                                      'accuracy':regionalPredictSet['accuracy'].tolist(), 'pred0':predColumns[0], \
                                      'pred1':predColumns[1], 'pred2':predColumns[2], 'conf0': confColumns[0], \
                                      'conf1': confColumns[1], 'conf2': confColumns[2], 'regionalFBScore': [fbscoreForSerialization]*len(regionalPredictSet)})

                results.to_csv(resultFilePath)



        count+=1
        if count%10==0:

            print '{} : total time {} s.'.format(count, time.time()-startTime)
            print 'Average training Time: ', sum(trainingTimes)/len(trainingTimes)
            print 'Average prediction Time: ', sum(predictionTimes)/len(predictionTimes)
            print 'Average FB Score', np.mean(FBscores)
            print 
            
            startTime=time.time()
            trainingTimes=[]
            predictionTimes=[]


    print 
    print np.mean(accuracies)
    print np.var(accuracies)
    print
    print np.mean(FBscores)
    print testNum
    print 'done'
    
    
    
    

In [53]:
train_test(modelSerialization=False, resultSerialization=True, predictSet=valid)

region 0: 0,0 accuracy: 0.667797568561,  fbAccu: 0.579681462633, confidence: 0.597089906701:
region 1: 0,1 accuracy: 0.653834308593,  fbAccu: 0.564931731855, confidence: 0.584311364336:
region 2: 0,2 accuracy: 0.630360531309,  fbAccu: 0.525932953827, confidence: 0.547810246679:
region 3: 0,3 accuracy: 0.678403105073,  fbAccu: 0.586914333241, confidence: 0.580571111727:
region 4: 0,4 accuracy: 0.679130874179,  fbAccu: 0.580133063837, confidence: 0.630763516928:
region 5: 0,5 accuracy: 0.67328,  fbAccu: 0.593013333333, confidence: 0.62491904:
region 6: 0,6 accuracy: 0.728243021346,  fbAccu: 0.614668856048, confidence: 0.606283661741:
region 7: 0,7 accuracy: 0.667805519054,  fbAccu: 0.565746824354, confidence: 0.595761103811:
region 8: 0,8 accuracy: 0.642563399589,  fbAccu: 0.540495773361, confidence: 0.572132282385:
region 9: 0,9 accuracy: 0.664014146773,  fbAccu: 0.576186265841, confidence: 0.574706454465:
10 : total time 404.564847946 s.
Average training Time:  23.9833799124
Average pr

KeyboardInterrupt: 

In [12]:
def makeSubmission(resultFolder):
    results=[]
    for f in os.listdir(resultFolder):
        if f.endswith('.rst'):
            fi=resultFolder+'/'+f
            results.append(pd.read_csv(fi))
    results=pd.concat(results)
        
    results.sort(['originalIndex'], inplace=True)
    return results

In [13]:
results=makeSubmission('../submissions/xgb-0.03-50Trees/')
predictions=results[['pred0','pred1','pred2']]
submit=pd.DataFrame()
# submit.loc[:,'row_id']=np.arange(len(predictions))
submit.loc[:,'place_id']=predictions[['pred0', 'pred1', 'pred2']].apply(lambda x: ' '.join([str(nx) for nx in x]), axis=1)
submit.loc[:,'row_id']=np.arange(len(predictions))


/home/zhenjie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [14]:
submit=submit[['row_id','place_id' ]]
submit.set_index('row_id', inplace=True)
submit.to_csv('../submissions/xgb0.03_50Trees.csv')


In [267]:
p300=pd.read_csv('../submissions/rf300Trees10_250.csv').head()
p50=pd.read_csv('../submissions/rf50Trees10_250.csv').head()
p300
p50

,row_id,place_id
0,0,8017323210 1985125281 4393146716
1,1,2465239230 5437803702 2921898487
2,2,2516481553 6692804575 7862615088
3,3,7995458948 3243409743 8393706174
4,4,8711861736 2766376680 9619154293


,row_id,place_id
0,0,8017323210 1985125281 6131996960
1,1,2465239230 5801740503 4223683383
2,2,2516481553 6692804575 7862615088
3,3,7995458948 3243409743 5345410711
4,4,8711861736 2766376680 9619154293


In [238]:
others=pd.read_csv('../submissions/rf_submission_2016-06-03-19-57.csv')

In [189]:
results=makeSubmission('rf-Tue Jun 14 18:18:14 2016')
predictions=results[['pred0','pred1','pred2']].values

realXMin=0
realXMax=10.1
realYMin=0
realYMax=10.1

real=_train_[int(len(_train_)*trainRatio):]

real=real[(real['x']<realXMax) & (real['x']>=realXMin) & (real['y']<realYMax) & (real['y']>=realYMin)]
real=real[['place_id']]

print len(real)
print len(predictions)

# print len(predictions), len(real), results['originalIndex'].value_counts()

totalHit=sum([real['place_id'].iloc[i] in predictions[i] for i in xrange(len(predictions))])
score=0
score+=1.0*sum([real['place_id'].iloc[i] in predictions[i][:1] for i in xrange(len(predictions))])
score+=1.0/2*sum([real['place_id'].iloc[i] in predictions[i][1:2] for i in xrange(len(predictions))])
score+=1.0/3*sum([real['place_id'].iloc[i] in predictions[i][2:3] for i in xrange(len(predictions))])

print 1.0*totalHit/len(predictions)
print score/len(predictions)



/home/zhenjie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


5730846
5730846
0.570895466394
0.494175961688


In [23]:
real.head()

results[results['originalIndex']==934546]

23294733    9851590985
23294779    1983039994
23294813    3915691710
23295061    4751915737
23295121    7372147589
Name: place_id, dtype: int64

,Unnamed: 0,accuracy,conf0,conf1,conf2,originalIndex,pred0,pred1,pred2,x,y
306,306,65,0.44,0.26,0.20,934546,3137026172,5758936842,8767363147,0.3569,0.84
393,393,65,0.44,0.22,0.06,934546,3137026172,5758936842,8675906672,0.3569,0.84


In [29]:
r=pd.read_csv('rf-Sun Jun 12 12:17:04 2016/0000-0.0-1.0-0.0-0.04.rst')

In [30]:
r.head()

,Unnamed: 0,accuracy,confidence,prediction,x,y
0,23295372,3,"(0.76000000000000001, 0.14000000000000001, 0.0...","(9516247724, 7965058889, 8813608705)",0.4866,0.0119
1,23295972,169,"(0.28000000000000003, 0.20000000000000001, 0.12)","(4941765890, 1342336464, 2797440100)",0.8878,0.0279
2,23296020,8,"(1.0, 0.0, 0.0)","(1425204074, 9973067176, 9922884570)",0.9640,0.0213
3,23297600,77,"(1.0, 0.0, 0.0)","(9019790086, 9973067176, 9922884570)",0.9911,0.0206
4,23299123,16,"(0.32000000000000001, 0.29999999999999999, 0.1...","(2718702529, 9727638738, 7937617850)",0.1835,0.0201
